In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
# เราจะลองใช้โมเดลพื้นฐานที่ตีความง่าย
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report

# --- 1. โหลดข้อมูลที่จัดกลุ่มแล้ว ---
print("--- 1. Loading Clustered Data ---")
path_base = "../data/PROCESSED/"
clustered_df_path = path_base + "master_features_clustered.csv"

try:
    df_model = pd.read_csv(clustered_df_path)
    print(f"Successfully loaded: {clustered_df_path}")
except Exception as e:
    print(f"Error loading file: {e}")

# --- 2. กำหนด X (Features) และ y (Target) ---
print("\n--- 2. Defining X and y ---")

# y (Target) คือคอลัมน์ 'cluster'
y = df_model['cluster']

# X (Features) คือคอลัมน์ที่เราใช้ใน K-Means
# (คัดลอก List มาจาก Notebook ที่แล้ว)
one_hot_cols = [col for col in df_model.columns if 'group_' in col]
one_hot_cols = sorted(list(set(one_hot_cols))) # จัดการตัวซ้ำ

numeric_features = [
    'AREA', 'population', 'canal_count', 'flood_point_count',
    'floodgate_count', 'คะแนนรวม', 'จำนวนจุดเสี่ยง', 'จำนวนจุดเฝ้าระวังน้ำท่วม',
    'rain_max_24h', 'rain_sum_24h', 'rain_days_heavy',
    'population_density', 'canal_density', 'flood_point_density',
    'floodgate_density', 'pump_per_capita', 'pump_readiness_ratio'
]
features_list = numeric_features + one_hot_cols

# ตรวจสอบว่าคอลัมน์มีอยู่จริง
features_list = [col for col in features_list if col in df_model.columns]

X = df_model[features_list]

print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")

# --- 3. Preprocessing (Log Transform & Scaling) ---
# เราต้องทำเหมือนกับตอนทำ K-Means เป๊ะๆ
print("\n--- 3. Preprocessing X features ---")

# 3a. Log Transformation
skewed_features = [
    'AREA', 'population', 'canal_count', 'flood_point_count', 
    'floodgate_count', 'rain_max_24h', 'rain_sum_24h', 
    'population_density', 'canal_density', 'flood_point_density',
    'floodgate_density', 'pump_per_capita'
]

for col in skewed_features:
    if col in X.columns:
        X[col] = np.log1p(X[col])

# 3b. Standard Scaling
# (สำคัญ: ในโลกจริง เราควร .fit(X_train) และ .transform(X_test)
# แต่สำหรับข้อมูล 50 แถวนี้ เราจะ Scale ทั้งหมดไปก่อนเพื่อความง่าย)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print("Features transformed (log) and scaled.")

# --- 4. Train/Test Split ---
print("\n--- 4. Performing Train/Test Split ---")
# เราจะแบ่งข้อมูล 70% เป็น Train, 30% เป็น Test
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, 
    test_size=0.3,    # แบ่ง 30% ไปเป็น Test
    random_state=42,  # ล็อคผลลัพธ์
    stratify=y        # (สำคัญ) พยายามเกลี่ยให้มี cluster 0,1,2,3 ใน test set
)

print(f"Training set size: {X_train.shape[0]} samples")
print(f"Test set size: {X_test.shape[0]} samples")

# --- 5. Train Model (Decision Tree) ---
print("\n--- 5. Training Decision Tree Classifier ---")
model = DecisionTreeClassifier(random_state=42)
model.fit(X_train, y_train)

print("Model trained.")

# --- 6. Test Model (Evaluate) ---
print("\n--- 6. Evaluating Model on Test Set ---")
y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy * 100:.2f}%")

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# (หมายเหตุ: ด้วยข้อมูล 50 แถว, test set จะมีแค่ 15 แถว, 
# Accuracy อาจจะผันผวนสูงมาก นี่เป็นแค่การสาธิตขั้นตอนครับ)

--- 1. Loading Clustered Data ---
Successfully loaded: ../data/PROCESSED/master_features_clustered.csv

--- 2. Defining X and y ---
X shape: (50, 29)
y shape: (50,)

--- 3. Preprocessing X features ---
Features transformed (log) and scaled.

--- 4. Performing Train/Test Split ---
Training set size: 35 samples
Test set size: 15 samples

--- 5. Training Decision Tree Classifier ---
Model trained.

--- 6. Evaluating Model on Test Set ---
Model Accuracy: 100.00%

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         2
           1       1.00      1.00      1.00         7
           2       1.00      1.00      1.00         3
           3       1.00      1.00      1.00         3

    accuracy                           1.00        15
   macro avg       1.00      1.00      1.00        15
weighted avg       1.00      1.00      1.00        15



/var/folders/89/hx457xt54dl3cdz1fssyz4tr0000gn/T/ipykernel_37003/2184726496.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = np.log1p(X[col])


In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report

# --- Import Models ---
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier  # <-- โมเดลที่ 2
from sklearn.linear_model import LogisticRegression # <-- โมเดลที่ 3

# --- 1. โหลดข้อมูลที่จัดกลุ่มแล้ว ---
print("--- 1. Loading Cluttered Data ---")
path_base = "../data/PROCESSED/"
clustered_df_path = path_base + "master_features_clustered.csv"

try:
    df_model = pd.read_csv(clustered_df_path)
    print(f"Successfully loaded: {clustered_df_path}")
except Exception as e:
    print(f"Error loading file: {e}")

# --- 2. กำหนด X (Features) และ y (Target) ---
print("\n--- 2. Defining X and y ---")
y = df_model['cluster']

# X (Features)
one_hot_cols = [col for col in df_model.columns if 'group_' in col]
one_hot_cols = sorted(list(set(one_hot_cols)))
numeric_features = [
    'AREA', 'population', 'canal_count', 'flood_point_count',
    'floodgate_count', 'คะแนนรวม', 'จำนวนจุดเสี่ยง', 'จำนวนจุดเฝ้าระวังน้ำท่วม',
    'rain_max_24h', 'rain_sum_24h', 'rain_days_heavy',
    'population_density', 'canal_density', 'flood_point_density',
    'floodgate_density', 'pump_per_capita', 'pump_readiness_ratio'
]
features_list = numeric_features + one_hot_cols
features_list = [col for col in features_list if col in df_model.columns]

# (แก้ SettingWithCopyWarning)
X = df_model[features_list].copy() 
print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")

# --- 3. Preprocessing (Log Transform & Scaling) ---
print("\n--- 3. Preprocessing X features ---")

# 3a. Log Transformation
skewed_features = [
    'AREA', 'population', 'canal_count', 'flood_point_count', 
    'floodgate_count', 'rain_max_24h', 'rain_sum_24h', 
    'population_density', 'canal_density', 'flood_point_density',
    'floodgate_density', 'pump_per_capita'
]

for col in skewed_features:
    if col in X.columns:
        X[col] = np.log1p(X[col])

# 3b. Standard Scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
print("Features transformed (log) and scaled.")

# --- 4. Train/Test Split ---
print("\n--- 4. Performing Train/Test Split ---")
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, 
    test_size=0.3,
    random_state=42,
    stratify=y
)
print(f"Training set size: {X_train.shape[0]} samples")
print(f"Test set size: {X_test.shape[0]} samples")


# ==========================================================
# --- 5. Model Training & Comparison ---
# ==========================================================

# --- Model 1: Decision Tree (Baseline) ---
print("\n--- 1. Decision Tree Classifier ---")
model_dt = DecisionTreeClassifier(random_state=42)
model_dt.fit(X_train, y_train)
y_pred_dt = model_dt.predict(X_test)
accuracy_dt = accuracy_score(y_test, y_pred_dt)
print(f"Decision Tree Accuracy: {accuracy_dt * 100:.2f}%")
print(classification_report(y_test, y_pred_dt, zero_division=0))


# --- Model 2: Random Forest ---
print("\n--- 2. Random Forest Classifier ---")
# n_estimators=100 คือสร้าง 100 ต้นไม้
model_rf = RandomForestClassifier(n_estimators=100, random_state=42) 
model_rf.fit(X_train, y_train)
y_pred_rf = model_rf.predict(X_test)
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print(f"Random Forest Accuracy: {accuracy_rf * 100:.2f}%")
print(classification_report(y_test, y_pred_rf, zero_division=0))


# --- Model 3: Logistic Regression ---
print("\n--- 3. Logistic Regression ---")
# max_iter=1000 เพื่อให้แน่ใจว่าโมเดลเรียนรู้จนจบ
model_lr = LogisticRegression(random_state=42, max_iter=1000)
model_lr.fit(X_train, y_train)
y_pred_lr = model_lr.predict(X_test)
accuracy_lr = accuracy_score(y_test, y_pred_lr)
print(f"Logistic Regression Accuracy: {accuracy_lr * 100:.2f}%")
print(classification_report(y_test, y_pred_lr, zero_division=0))

# --- 6. Summary ---
print("\n--- Model Comparison Summary ---")
print(f"Decision Tree Accuracy:     {accuracy_dt * 100:.2f}%")
print(f"Random Forest Accuracy:     {accuracy_rf * 100:.2f}%")
print(f"Logistic Regression Accuracy: {accuracy_lr * 100:.2f}%")

--- 1. Loading Cluttered Data ---
Successfully loaded: ../data/PROCESSED/master_features_clustered.csv

--- 2. Defining X and y ---
X shape: (50, 29)
y shape: (50,)

--- 3. Preprocessing X features ---
Features transformed (log) and scaled.

--- 4. Performing Train/Test Split ---
Training set size: 35 samples
Test set size: 15 samples

--- 1. Decision Tree Classifier ---
Decision Tree Accuracy: 100.00%
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         2
           1       1.00      1.00      1.00         7
           2       1.00      1.00      1.00         3
           3       1.00      1.00      1.00         3

    accuracy                           1.00        15
   macro avg       1.00      1.00      1.00        15
weighted avg       1.00      1.00      1.00        15


--- 2. Random Forest Classifier ---
Random Forest Accuracy: 100.00%
              precision    recall  f1-score   support

           0       1.00      1.00  